# 라이브러리 설치하기 

In [2]:
!pip install scikit-learn==1.3.1 pandas azure-storage-blob==12.9.0 kserve==0.11.2

  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.5/356.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.5 MB/s eta 0:00:00
Using cached typing_extensions-4.11.0-py3-none-any.whl (34 kB)
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_exten

# 모델 정의 및 학습 

In [1]:
import sklearn
sklearn.__version__

'1.3.1'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

# Create a random dataset
rng = np.random.RandomState(1)
X = np.sort(200 * rng.rand(100, 1) - 100, axis=0)
y = np.array([np.pi * np.sin(X).ravel(), np.pi * np.cos(X).ravel()]).T
y[::5, :] += 0.5 - rng.rand(20, 2)

# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_3 = DecisionTreeRegressor(max_depth=8)
regr_1.fit(X, y)
regr_2.fit(X, y)
regr_3.fit(X, y)

# Predict
X_test = np.arange(-100.0, 100.0, 0.01)[:, np.newaxis]
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)
y_3 = regr_3.predict(X_test)

In [3]:
y_3

array([[ 1.82569761,  2.64490159],
       [ 1.82569761,  2.64490159],
       [ 1.82569761,  2.64490159],
       ...,
       [-1.84319053, -0.55258289],
       [-1.84319053, -0.55258289],
       [-1.84319053, -0.55258289]])

# 학습된 모델 저장하기 

In [4]:
import joblib

In [5]:
import os
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',   # Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0'    # If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})  

bucket_name="kserve-test"
model_path="model.joblib"
joblib.dump(regr_3, model_path)

['model.joblib']

In [7]:
!mc mb kubeflow/kserve-test

]11;?\Bucket created successfully `kubeflow/kserve-test`.


In [8]:
!mc ls kubeflow

]11;?\[2024-04-28 01:31:10 UTC]     0B kserve-test/
[2024-04-27 02:55:11 UTC]     0B mlpipeline/
[2024-04-27 11:22:21 UTC]     0B new-bucket/
[2024-04-27 11:58:02 UTC]     0B test-client/
[2024-04-27 12:09:25 UTC]     0B test-sdk/


In [9]:
!mc cp model.joblib kubeflow/kserve-test/sklearn-test/model.joblib

...del.joblib: 13.71 KiB / 13.71 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 542.96 KiB/s 0s

In [10]:
!mc ls kubeflow/kserve-test/sklearn-test

]11;?\[2024-04-28 01:31:56 UTC]  14KiB STANDARD model.joblib
